In [1]:
import time

import datetime

import netCDF4
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
ds = netCDF4.Dataset('/Users/baart_f/data/odv/data_from_SDN_2017-11_TS_profiles_non-restricted_med.nc', 'a')

In [3]:
def compute_date_time_ncattrs(ds):
    """compute extra netcdf attributes to make the file a bit more optimal"""
    
    date_nums = ds.variables['date_time'][:]
    date_unit = ds.variables['date_time'].units
    
    # don't convert all nums to date, that's a bit slow, because here we explicitly use a gregorian calendar
    
    def is_valid_date(maybe_datetime):
        if maybe_datetime.year < 1800:
            return False

        return True

    # not all date times are valid
    iter_valid_dates = (
        t 
        for t 
        in sorted(date_nums) 
        if is_valid_date(
            netCDF4.num2date(t, date_unit)
        )
    )
    
    # these are the actual (sorted values)
    iter_actual_dates = (
        t 
        for t 
        in sorted(date_nums) 
    )

    # pick the first
    t0_valid = next(iter_valid_dates)
    t0_actual = next(iter_actual_dates)
    
    # assume the last is valid
    assert is_valid_date(netCDF4.num2date(date_nums.max(), date_unit))
    
    ncattrs = {
        "actual_range": [t0_actual, date_nums.max()],
        "valid_range": [t0_valid, date_nums.max()]
    }
    return ncattrs


In [7]:
ncattrs = compute_date_time_ncattrs(ds)
date_time = ds.variables['date_time']
date_time.setncatts(ncattrs)
ds.sync()
